In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("../data/amazon_sales_2025_raw.csv", encoding="utf-8")
df.head()

,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,buy_box_availability,delivery_details,sustainability_badges,image_url,product_url,collected_at
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6 out of 5 stars,375,300+ bought in past month,89.68,basic variant price: 2.4GHz,$159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,"Delivery Mon, Sep 1",Carbon impact,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3 out of 5 stars,"2,457",6K+ bought in past month,9.99,basic variant price: nan,$15.99,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Fri, Aug 29",NaN,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6 out of 5 stars,"3,044",2K+ bought in past month,314.00,basic variant price: nan,$349.00,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Mon, Sep 1",NaN,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6 out of 5 stars,"35,882",10K+ bought in past month,NaN,basic variant price: $162.24,No Discount,Best Seller,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8 out of 5 stars,"28,988",10K+ bought in past month,NaN,basic variant price: $72.74,No Discount,No Badge,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29


## Quick Review 

In [3]:
print(f"The dataframe has {df.shape[0]} rows and {df.shape[1]} columns.")

The dataframe has 42675 rows and 16 columns.


In [4]:
print(f"The dataframe columns name: {df.columns}")

The dataframe columns name: Index(['title', 'rating', 'number_of_reviews', 'bought_in_last_month',
       'current/discounted_price', 'price_on_variant', 'listed_price',
       'is_best_seller', 'is_sponsored', 'is_couponed', 'buy_box_availability',
       'delivery_details', 'sustainability_badges', 'image_url', 'product_url',
       'collected_at'],
      dtype='object')


In [5]:
num_duplicates = df.duplicated().sum()
print(f"Number of duplicated rows: {num_duplicates}")

Number of duplicated rows: 0


In [6]:
df.describe()

,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,buy_box_availability,delivery_details,sustainability_badges,image_url,product_url,collected_at
count,42675,41651,41651,39458,30926,42675,42675,42675,42675,42675,28022,30955,3408,42675,40606,42675
unique,8808,31,4413,59,2576,3466,911,12,2,42,1,298,16,8038,40606,1559
top,"Duracell Coppertop 9V Battery, 6 Count (Pack o...",4.6 out of 5 stars,25,100+ bought in past month,29.99,basic variant price: nan,No Discount,No Badge,Organic,No Coupon,Add to cart,"Delivery Mon, Sep 1",Small Business,https://m.media-amazon.com/images/I/51Xr76m2WL...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
freq,744,6151,626,8801,825,21031,30364,40814,35664,40727,28022,6189,1341,1461,1,33


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   title                     42675 non-null  object
 1   rating                    41651 non-null  object
 2   number_of_reviews         41651 non-null  object
 3   bought_in_last_month      39458 non-null  object
 4   current/discounted_price  30926 non-null  object
 5   price_on_variant          42675 non-null  object
 6   listed_price              42675 non-null  object
 7   is_best_seller            42675 non-null  object
 8   is_sponsored              42675 non-null  object
 9   is_couponed               42675 non-null  object
 10  buy_box_availability      28022 non-null  object
 11  delivery_details          30955 non-null  object
 12  sustainability_badges     3408 non-null   object
 13  image_url                 42675 non-null  object
 14  product_url           

In [8]:
print("Number of missing values for each columns: ")
df.isna().sum()

Number of missing values for each columns: 


title                           0
rating                       1024
number_of_reviews            1024
bought_in_last_month         3217
current/discounted_price    11749
price_on_variant                0
listed_price                    0
is_best_seller                  0
is_sponsored                    0
is_couponed                     0
buy_box_availability        14653
delivery_details            11720
sustainability_badges       39267
image_url                       0
product_url                  2069
collected_at                    0
dtype: int64

## Data Cleaning and Transformation

### Clean `rating`  Columns

1. rating (Ürün Puanı)
Orijinal Format: "4.6 out of 5 stars" (object/string)
Hedef Format: 4.6 (float)
Yapılan İşlemler:

" out of 5 stars" ifadesi kaldırıldı
Sayısal değere (float) dönüştürüldü
Dönüştürülemeyen değerler NaN olarak bırakıldı

Sebep: Matematiksel işlemler yapabilmek (ortalama, filtreleme, vb.) için sayısal formata ihtiyacımız var.
Eksik Veri Durumu: 1024 eksik değer var (%2.4)

In [9]:
print("Rating sütununun ilk 20 benzersiz değeri:")
print(df['rating'].value_counts().head(20))
print("\n")

Rating sütununun ilk 20 benzersiz değeri:
rating
4.6 out of 5 stars    6151
4.4 out of 5 stars    5525
4.5 out of 5 stars    5359
4.7 out of 5 stars    4664
4.8 out of 5 stars    4230
4.3 out of 5 stars    2927
4.2 out of 5 stars    2837
4.1 out of 5 stars    1959
4.0 out of 5 stars    1465
3.9 out of 5 stars    1316
3.8 out of 5 stars    1083
5.0 out of 5 stars     995
4.9 out of 5 stars     704
3.6 out of 5 stars     666
3.7 out of 5 stars     604
3.2 out of 5 stars     363
3.5 out of 5 stars     242
3.4 out of 5 stars     216
3.0 out of 5 stars     148
2.0 out of 5 stars     143
Name: count, dtype: int64




In [10]:
# Birkaç örnek değer görelim
print("İlk 10 rating değeri:")
print(df['rating'].head(10))
print("\n")

İlk 10 rating değeri:
0    4.6 out of 5 stars
1    4.3 out of 5 stars
2    4.6 out of 5 stars
3    4.6 out of 5 stars
4    4.8 out of 5 stars
5    4.6 out of 5 stars
6    4.5 out of 5 stars
7    4.6 out of 5 stars
8    4.0 out of 5 stars
9    4.7 out of 5 stars
Name: rating, dtype: object




In [11]:
# Eksik değerleri görelim
print(f"Eksik rating sayısı: {df['rating'].isna().sum()}")

Eksik rating sayısı: 1024


In [12]:
df['rating'] = df['rating'].str.replace(r"out of 5 stars", "").str.strip().astype(float)

In [13]:
rating_counts = df['rating'].value_counts(dropna=False)
print(rating_counts)

rating
4.6    6151
4.4    5525
4.5    5359
4.7    4664
4.8    4230
4.3    2927
4.2    2837
4.1    1959
4.0    1465
3.9    1316
3.8    1083
NaN    1024
5.0     995
4.9     704
3.6     666
3.7     604
3.2     363
3.5     242
3.4     216
3.0     148
2.0     143
2.7      15
1.5      15
3.3       6
2.8       4
1.0       4
2.4       3
2.9       2
3.1       2
2.3       1
2.5       1
2.6       1
Name: count, dtype: int64


### Clean `number_of_reviews`  Columns

In [14]:
print("Number of Reviews sütununun ilk 20 benzersiz değeri:")
print(df['number_of_reviews'].value_counts().head(20))
print("\n")

Number of Reviews sütununun ilk 20 benzersiz değeri:
number_of_reviews
25       626
1        524
8        478
17       409
30       409
76       380
93       357
6        350
9        329
125      325
2        317
84       309
7        292
14       279
29       272
182      270
5,257    254
155      247
45       231
4        226
Name: count, dtype: int64




In [15]:
print("Benzersiz değer örnekleri:")
print(df['number_of_reviews'].value_counts().head(15))
print("\n")

Benzersiz değer örnekleri:
number_of_reviews
25     626
1      524
8      478
17     409
30     409
76     380
93     357
6      350
9      329
125    325
2      317
84     309
7      292
14     279
29     272
Name: count, dtype: int64




In [16]:
print(f"Eksik deger sayisi: {df['number_of_reviews'].isna().sum()}")

Eksik deger sayisi: 1024


In [17]:
df['number_of_reviews'] = df['number_of_reviews'].str.replace(",", "").str.strip().astype(float)

In [18]:
reviews_counts = df['number_of_reviews'].value_counts(dropna=False)
print(reviews_counts)

number_of_reviews
NaN        1024
25.0        626
1.0         524
8.0         478
30.0        409
           ... 
7161.0        1
3396.0        1
65165.0       1
18638.0       1
1589.0        1
Name: count, Length: 4414, dtype: int64


### Clean `bought_in_last_month`  Columns

In [19]:
print("Bought in last month sütununun ilk 20 örneği:")
print(df['bought_in_last_month'].head(20))
print("\n")

Bought in last month sütununun ilk 20 örneği:
0      300+ bought in past month
1       6K+ bought in past month
2       2K+ bought in past month
3      10K+ bought in past month
4      10K+ bought in past month
5     100K+ bought in past month
6      10K+ bought in past month
7      10K+ bought in past month
8                            NaN
9      10K+ bought in past month
10    100K+ bought in past month
11     10K+ bought in past month
12     100+ bought in past month
13      1K+ bought in past month
14     500+ bought in past month
15    100K+ bought in past month
16     80K+ bought in past month
17    100K+ bought in past month
18    100K+ bought in past month
19     10K+ bought in past month
Name: bought_in_last_month, dtype: object




In [20]:
print("Benzersiz değer örnekleri:")
print(df['bought_in_last_month'].value_counts().head(20))
print("\n")


Benzersiz değer örnekleri:
bought_in_last_month
100+ bought in past month                                                                                                                                                       8801
50+ bought in past month                                                                                                                                                        5967
200+ bought in past month                                                                                                                                                       5645
No featured offers available                                                                                                                                                    4571
300+ bought in past month                                                                                                                                                       2842
500+ bought in past month                      

In [21]:
print(f"Eksik bought_in_last_month sayısı: {df['bought_in_last_month'].isna().sum()}")

Eksik bought_in_last_month sayısı: 3217


In [22]:
df['bought_in_last_month'] = df['bought_in_last_month'].str.replace("+ bought in past month","").str.strip().str.replace("K","000")

df['bought_in_last_month'] = df['bought_in_last_month'].where(df['bought_in_last_month'].str.isdigit(),np.nan)

In [23]:
df['bought_in_last_month'] = (df['bought_in_last_month'].where(df['bought_in_last_month'].str.isdigit(), np.nan).astype('Int64'))

In [24]:
bought_in_last_month_counts = df['bought_in_last_month'].value_counts(dropna=False)
print(bought_in_last_month_counts)

bought_in_last_month
<NA>      10511
100        8801
50         5967
200        5645
300        2842
500        2351
1000       2084
400        1436
20000       772
2000        426
800         280
3000        249
600         229
10000       229
4000        196
5000        120
700         102
90000        92
6000         81
900          63
7000         49
9000         44
8000         36
30000        28
40000        14
100000       13
50000         6
70000         4
60000         3
80000         2
Name: count, dtype: Int64


### Clean `current/discounted_price` and `price_on_variant` Columns

In [25]:
print("current/discounted_price sütununun ilk 20 örneği:")
print(df['current/discounted_price'].head(20))
print(f"Eksik: {df['current/discounted_price'].isna().sum()}")
print("\n")

current/discounted_price sütununun ilk 20 örneği:
0      89.68
1       9.99
2     314.00
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8      16.99
9        NaN
10     14.99
11       NaN
12     29.99
13    249.99
14     69.95
15       NaN
16     39.97
17     14.49
18     16.98
19       NaN
Name: current/discounted_price, dtype: object
Eksik: 11749




In [26]:
print("Benzersiz değer örnekleri:")
print(df['current/discounted_price'].value_counts().head(20))
print("\n")

Benzersiz değer örnekleri:
current/discounted_price
29.99     825
119.99    695
13.99     585
24.99     502
49.99     478
59.99     474
49.95     471
25.47     461
85.99     441
34.99     377
179.99    332
699.99    296
19.69     283
79.99     280
89.99     273
269.99    241
129.99    238
69.95     237
199.99    229
139.99    224
Name: count, dtype: int64




In [27]:
print(f"Eksik deger sayısı: {df['current/discounted_price'].isna().sum()}")

Eksik deger sayısı: 11749


In [28]:
# Price on variant sütununu inceleyelim
print("Price on Variant sütununun ilk 20 örneği:")
print(df['price_on_variant'].head(20))
print("\n")


Price on Variant sütununun ilk 20 örneği:
0      basic variant price: 2.4GHz
1         basic variant price: nan
2         basic variant price: nan
3     basic variant price: $162.24
4      basic variant price: $72.74
5      basic variant price: $99.95
6      basic variant price: $88.11
7      basic variant price: $23.04
8         basic variant price: nan
9     basic variant price: $284.05
10        basic variant price: nan
11    basic variant price: $880.95
12        basic variant price: nan
13        basic variant price: nan
14        basic variant price: nan
15     basic variant price: $17.45
16     basic variant price: $38.86
17        basic variant price: nan
18     basic variant price: $13.88
19     basic variant price: $33.51
Name: price_on_variant, dtype: object




In [29]:
# Benzersiz değer örnekleri
print("Benzersiz değer örnekleri:")
print(df['price_on_variant'].value_counts().head(20))
print("\n")

Benzersiz değer örnekleri:
price_on_variant
basic variant price: nan                     21031
basic variant price: 16 Count (Pack of 1)      582
basic variant price: $9.99                     324
basic variant price: $119.95                   298
basic variant price: $169.99                   264
basic variant price: $29.99                    217
basic variant price: $349.00                   211
basic variant price: $5.00                     209
basic variant price: $59.24                    205
basic variant price: $224.00                   204
basic variant price: $20.63                    204
basic variant price: $216.66                   204
basic variant price: $352.12                   204
basic variant price: $3.98                     204
basic variant price: $19.82                    202
basic variant price: $3,797.85                 200
basic variant price: $57.17                    195
basic variant price: $201.02                   193
basic variant price: $66.99           

In [30]:
# Eksik değerleri görelim
print(f"Eksik price_on_variant sayısı: {df['price_on_variant'].isna().sum()}")

Eksik price_on_variant sayısı: 0


In [31]:
df['price_on_variant'] = df['price_on_variant'].str.split(":").str.get(1)

df.loc[~df['price_on_variant'].str.contains(r'\$', na=False), 'price_on_variant'] = np.nan

df['price_on_variant'] = df['price_on_variant'].str.strip().str.split(" ").str.get(0)

In [32]:
df['current/discounted_price'] = df['current/discounted_price'].fillna(df['price_on_variant'])

In [33]:
df['current/discounted_price'] = df['current/discounted_price'].str.replace(r"\$", "", regex=True).str.replace(r",", "").astype(float)

In [34]:
print(f"Remaining missing values in 'current/discounted_price': {df['current/discounted_price'].isna().sum()}")

Remaining missing values in 'current/discounted_price': 2062


### Clean `listed_price`  Columns

In [35]:
# Listed price sütununu inceleyelim
print("Listed Price sütununun ilk 20 örneği:")
print(df['listed_price'].head(20))
print("\n")

Listed Price sütununun ilk 20 örneği:
0         $159.00
1          $15.99
2         $349.00
3     No Discount
4     No Discount
5     No Discount
6     No Discount
7     No Discount
8     No Discount
9     No Discount
10    No Discount
11    No Discount
12    No Discount
13    No Discount
14    No Discount
15    No Discount
16    No Discount
17    No Discount
18    No Discount
19    No Discount
Name: listed_price, dtype: object




In [36]:
# Eksik değerleri görelim
print(f"Eksik listed_price sayısı: {df['listed_price'].isna().sum()}")

Eksik listed_price sayısı: 0


In [37]:
# Benzersiz değer örnekleri
print("Benzersiz değer örnekleri:")
print(df['listed_price'].value_counts().head(20))
print("\n")

Benzersiz değer örnekleri:
listed_price
No Discount    30364
$79.99           500
$29.99           406
$16.61           360
$23.26           283
$43.99           241
$23.99           232
$25.99           210
$14.29           207
$27.33           205
$451.99          204
$199.99          191
$99.99           189
$39.99           188
$59.95           185
$179.99          184
$109.99          183
$329.00          181
$149.99          175
$199.95          174
Name: count, dtype: int64




In [38]:
df['listed_price'] = df['listed_price'].str.replace("$", "").str.replace(",","").str.strip()
df['listed_price'] = df['listed_price'].replace("No Discount", np.nan)

In [39]:
num_missing_listed = df['listed_price'].isna().sum()
print(f"Number of missing values in 'listed_price': {num_missing_listed}")

Number of missing values in 'listed_price': 30364


In [40]:
df['listed_price'] = df['listed_price'].fillna(df['current/discounted_price'])

In [41]:
df['listed_price'] = df['listed_price'].astype(float)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     42675 non-null  object 
 1   rating                    41651 non-null  float64
 2   number_of_reviews         41651 non-null  float64
 3   bought_in_last_month      32164 non-null  Int64  
 4   current/discounted_price  40613 non-null  float64
 5   price_on_variant          20071 non-null  object 
 6   listed_price              40613 non-null  float64
 7   is_best_seller            42675 non-null  object 
 8   is_sponsored              42675 non-null  object 
 9   is_couponed               42675 non-null  object 
 10  buy_box_availability      28022 non-null  object 
 11  delivery_details          30955 non-null  object 
 12  sustainability_badges     3408 non-null   object 
 13  image_url                 42675 non-null  object 
 14  produc

In [43]:
df

,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,buy_box_availability,delivery_details,sustainability_badges,image_url,product_url,collected_at
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6,375.0,300,89.68,NaN,159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,"Delivery Mon, Sep 1",Carbon impact,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3,2457.0,6000,9.99,NaN,15.99,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Fri, Aug 29",NaN,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6,3044.0,2000,314.00,NaN,349.00,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Mon, Sep 1",NaN,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6,35882.0,10000,162.24,$162.24,162.24,Best Seller,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8,28988.0,10000,72.74,$72.74,72.74,No Badge,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42670,"Elgato 4K Pro, Internal Capture Card: 8K60 Pas...",5.0,1.0,100,195.99,NaN,195.99,No Badge,Organic,No Coupon,NaN,"FREE delivery Thu, Sep 4Or fastest delivery Tu...",NaN,https://m.media-amazon.com/images/I/51KCB+egEs...,/Elgato-4K-Pro-Internal-Capture/dp/B0DLR3WQWR/...,2025-08-30 19:56:33
42671,"Arlo Essential Spotlight Camera, Wireless Secu...",4.2,20.0,200,89.99,$25.00,89.99,No Badge,Organic,Save $25.00 with coupon,Add to cart,"FREE delivery Thu, Sep 4Or fastest delivery Mo...",NaN,https://m.media-amazon.com/images/I/51jV+o1LZE...,/Arlo-Essential-Spotlight-Camera-Surveillance/...,2025-08-30 19:56:33
42672,"GIGABYTE - AORUS FO32U2-32"" QD OLED Gaming Mon...",4.3,57.0,50,899.99,NaN,1099.99,Save 18%,Organic,No Coupon,Add to cart,"FREE delivery Thu, Sep 4Or fastest delivery We...",NaN,https://m.media-amazon.com/images/I/71ySPkNLkG...,/GIGABYTE-FO32U2-32-3840x2160-240Hz-FreeSync-A...,2025-08-30 19:56:33
42673,Monoprice XLR Male to 1/4-Inch TRS Male Cable ...,4.7,7102.0,500,10.39,NaN,15.98,No Badge,Organic,No Coupon,Add to cart,"FREE delivery Thu, Sep 4 on $35 of items shipp...",NaN,https://m.media-amazon.com/images/I/411c0JFJ79...,/Monoprice-XLR-Male-4-Inch-Cable/dp/B001UJEKZ6...,2025-08-30 19:56:33


### Clean `is_best_seller`  Columns

In [44]:
df["is_best_seller"].unique()

array(['No Badge', 'Best Seller', "Amazon's", 'Limited time deal',
       'Save 30%', 'Save 12%', 'Save 9%', 'Save 17%', 'Save 77%',
       'Ends in', 'Save 10%', 'Save 18%'], dtype=object)

In [45]:
df["is_best_seller"] = df["is_best_seller"].apply(lambda x: "Best Seller" if str(x).strip().lower() == "best seller" else "No Badge")
df["is_best_seller"].value_counts()

is_best_seller
No Badge       42400
Best Seller      275
Name: count, dtype: int64

### Clean `is_sponsored`  Columns

In [46]:
df["is_sponsored"].unique()

array(['Sponsored', 'Organic'], dtype=object)

In [47]:
df["is_sponsored"].nunique()

2

### Clean `is_couponed`  Columns

In [48]:
df["is_couponed"].unique()

array(['Save 15%  with coupon', 'No Coupon', 'Save 50%  with coupon',
       'Save $16.00  with coupon', 'Save $10.00  with coupon',
       'Save 10%  with coupon', 'Save $20.00  with coupon',
       'Save $4.00  with coupon', 'Save $0.33  with coupon',
       'Save $18.00  with coupon', 'Save 5%  with coupon',
       'Save $2.00  with coupon', 'Save $15.00  with coupon',
       'Save 6%  with coupon', 'Save $8.00  with coupon',
       'Save $40.00  with coupon', 'Save $1.50  with coupon',
       'Save 7%  with coupon', 'Save 20%  with coupon',
       'Save $1.00  with coupon', 'Save $5.00  with coupon',
       'Save $30.00  with coupon', 'Save $12.00  with coupon',
       'Save 25%  with coupon', 'Save $133.00  with coupon',
       'Save $50.00  with coupon', 'Save $11.00  with coupon',
       'Save 30%  with coupon', 'Save $3.00  with coupon',
       'Save 40%  with coupon', 'Save $25.00  with coupon',
       'Save $13.00  with coupon',
       'Save 25%  with coupon (some sizes/color

In [49]:
df["is_couponed"].nunique()

42

In [50]:

def extract_coupon_value_str(x):
    text = str(x)
    if "no coupon" in text.lower():
        return "No Coupon"
    if "%" in text:
        match = re.search(r"(\d+\.?\d*)%", text)
        return match.group(1) if match else "No Coupon"
    if "$" in text:
        match = re.search(r"\$(\d+\.?\d*)", text)
        return match.group(1) if match else "No Coupon"
    return "No Coupon"

df["is_couponed"] = df["is_couponed"].apply(extract_coupon_value_str)

df["is_couponed"].head(20)

0            15
1     No Coupon
2     No Coupon
3     No Coupon
4     No Coupon
5     No Coupon
6     No Coupon
7     No Coupon
8            50
9     No Coupon
10    No Coupon
11    No Coupon
12    No Coupon
13    No Coupon
14    No Coupon
15    No Coupon
16    No Coupon
17    No Coupon
18    No Coupon
19    No Coupon
Name: is_couponed, dtype: object

### Clean `buy_box_availability`  Columns

In [51]:
df["buy_box_availability"].unique()

array(['Add to cart', nan], dtype=object)

### Clean `sustainability_badges` Column

In [52]:
df["sustainability_badges"].unique()

array(['Carbon impact', nan, 'Energy efficiency',
       'Safer chemicals +1 more', 'Small Business',
       '1 sustainability certification', 'Works with Alexa',
       'Manufacturing practices', 'Forestry practices', 'Alexa Built-in',
       'Energy efficiency +3 more', 'Recycled materials',
       'Safer chemicals +2 more', 'Energy efficiency +1 more',
       'Recycled materials +3 more', 'Recycled materials +2 more',
       'Made in Italy'], dtype=object)

In [53]:
df["sustainability_badges"].nunique()

16

## Feature Engineering

### Discount Rate

In [54]:
df["discount_rate"] = ((df["listed_price"] - df["current/discounted_price"]) / df["listed_price"]) * 100


### Price Segment

In [55]:
df["price_segment"] = pd.qcut(df["current/discounted_price"], q=3, labels=["Low", "Medium", "High"])

### Has Discount (Binary)

In [56]:
df["has_discount"] = (df["listed_price"] > df["current/discounted_price"]).astype(int)

In [57]:
df['has_discount'].head()

0    1
1    1
2    1
3    0
4    0
Name: has_discount, dtype: int64

### Time

In [58]:
df["collected_at"] = pd.to_datetime(df["collected_at"], errors="coerce")
df["collection_day"] = df["collected_at"].dt.day
df["collection_month"] = df["collected_at"].dt.month
df["collection_week"] = df["collected_at"].dt.isocalendar().week


### Category by Title

In [59]:
category_map = {
    "Smartphones": [
        "iphone", "samsung", "galaxy", "pixel", "smartphone", "android"
    ],
    "Laptops & Computers": [
        "laptop", "macbook", "chromebook", "notebook", "desktop", "computer", "imac"
    ],
    "Tablets & iPads": [
        "ipad", "tablet", "surface"
    ],
    "Headphones & Earbuds": [
        "earbuds", "earbud", "headphone", "headphones", "airpods", "beats", "jbl", "sony", "bose"
    ],
    "Speakers & Audio": [
        "speaker", "soundbar", "bluetooth speaker", "roam", "move", "sonos"
    ],
    "Printers & Ink": [
        "printer", "ink", "toner", "cartridge", "laserjet", "officejet", "envy", "pixma", "ecotank"
    ],
    "Office Supplies": [
        "paper", "pen", "pencil", "folder", "tape", "binder", "stapler", "marker", "label", "calculator", "desk", "chair"
    ],
    "Storage Devices": [
        "ssd", "hdd", "hard drive", "flash drive", "usb drive", "sd card", "memory card"
    ],
    "Accessories & Cables": [
        "charger", "charging", "cable", "adapter", "dock", "hub", "stand", "mount", "case"
    ],
    "Networking & Routers": [
        "router", "wifi", "modem", "ethernet", "switch", "repeater", "tp-link", "netgear", "deco"
    ],
    "Gaming": [
        "xbox", "playstation", "nintendo", "controller", "joystick", "gaming"
    ],
    "Home & Appliances": [
        "vacuum", "fan", "lamp", "appliance", "toaster", "fryer"
    ],
    "Batteries & Power": [
        "battery", "batteries", "duracell", "energizer", "power strip", "surge protector"
    ],
    "Cameras & Drones": [
        "camera", "drone", "gimbal", "osmo", "instax", "kodak", "dji"
    ],
    "Smartwatch & Fitness": [
        "fitbit", "garmin", "watch", "smartwatch", "band", "tracker"
    ],
    "Smart Home & Security": [
        "doorbell", "security camera", "ring", "blink", "nest", "myq", "smart home"
    ],
    "Other": []
}


In [60]:
#unique_titles = df["title"].dropna().unique()

#with open("unique_titles.txt", "w", encoding="utf-8") as f:
#    for title in unique_titles:
#       f.write(str(title).strip() + "\n")


In [61]:
def extract_category(title):
    title_lower = str(title).lower()
    for category, keywords in category_map.items():
        if any(word in title_lower for word in keywords):
            return category
    return "Other"

df["product_category"] = df["title"].map(extract_category)


In [62]:
df["product_category"].value_counts(normalize=True).head(15)


product_category
Other                   0.182191
Laptops & Computers     0.121453
Smartphones             0.108377
Printers & Ink          0.090568
Headphones & Earbuds    0.082999
Accessories & Cables    0.072736
Batteries & Power       0.072267
Speakers & Audio        0.047920
Cameras & Drones        0.042671
Networking & Routers    0.036204
Storage Devices         0.032056
Gaming                  0.032033
Office Supplies         0.025518
Tablets & iPads         0.018559
Home & Appliances       0.014833
Name: proportion, dtype: float64

## Rename Columns

In [63]:
df.rename(columns={
    "title": "product_title",
    "rating": "product_rating",
    "number_of_reviews": "review_count",
    "bought_in_last_month": "sales_last_month",
    "current/discounted_price": "discounted_price",
    "price_on_variant": "variant_price",
    "listed_price": "original_price",
    "is_best_seller": "best_seller",
    "is_sponsored": "sponsored",
    "is_couponed": "coupon_available",
    "buy_box_availability": "buy_box_status",
    "delivery_details": "delivery_info",
    "sustainability_badges": "eco_badge",
    "image_url": "product_image_url",
    "product_url": "product_url",
    "collected_at": "collected_date"
}, inplace=True)


## Review After Data Cleaning

In [64]:
df.head()

,product_title,product_rating,review_count,sales_last_month,discounted_price,variant_price,original_price,best_seller,sponsored,coupon_available,...,product_image_url,product_url,collected_date,discount_rate,price_segment,has_discount,collection_day,collection_month,collection_week,product_category
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6,375.0,300,89.68,NaN,159.00,No Badge,Sponsored,15,...,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29,43.597484,Medium,1,21,8,34,Smartphones
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3,2457.0,6000,9.99,NaN,15.99,No Badge,Sponsored,No Coupon,...,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29,37.523452,Low,1,21,8,34,Smartphones
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6,3044.0,2000,314.00,NaN,349.00,No Badge,Sponsored,No Coupon,...,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29,10.028653,High,1,21,8,34,Smartphones
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6,35882.0,10000,162.24,$162.24,162.24,Best Seller,Organic,No Coupon,...,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29,0.000000,High,0,21,8,34,Headphones & Earbuds
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8,28988.0,10000,72.74,$72.74,72.74,No Badge,Organic,No Coupon,...,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29,0.000000,Medium,0,21,8,34,Smartphones


In [65]:
df.shape

(42675, 23)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42675 entries, 0 to 42674
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   product_title      42675 non-null  object        
 1   product_rating     41651 non-null  float64       
 2   review_count       41651 non-null  float64       
 3   sales_last_month   32164 non-null  Int64         
 4   discounted_price   40613 non-null  float64       
 5   variant_price      20071 non-null  object        
 6   original_price     40613 non-null  float64       
 7   best_seller        42675 non-null  object        
 8   sponsored          42675 non-null  object        
 9   coupon_available   42675 non-null  object        
 10  buy_box_status     28022 non-null  object        
 11  delivery_info      30955 non-null  object        
 12  eco_badge          3408 non-null   object        
 13  product_image_url  42675 non-null  object        
 14  produc

In [67]:
df.isna().sum()

product_title            0
product_rating        1024
review_count          1024
sales_last_month     10511
discounted_price      2062
variant_price        22604
original_price        2062
best_seller              0
sponsored                0
coupon_available         0
buy_box_status       14653
delivery_info        11720
eco_badge            39267
product_image_url        0
product_url           2069
collected_date           0
discount_rate         2062
price_segment         2062
has_discount             0
collection_day           0
collection_month         0
collection_week          0
product_category         0
dtype: int64